This notebook is the one to adapt to your data, the aim is to create `TGCA_Merged.csv` with the following columns:
- `t`: the time of event in years (the unit is relevant for the other notebooks)
- `e`: the event type (0 is censoring)
- `text`: the unstructured data
- `Grouping`: the splitting stucture (if different patient or hospital to consider together)
- Additional columns to represent structured data known to be predicitve of the survival outcome you aim to model (if any are available)
- Additional splitting

The example is based on the TCGA dataset

Extract all the data necessary for the analysis
To run, first download and extract in `data/` the following files:
- Report: https://github.com/tatonetti-lab/tcga-path-reports/blob/main/TCGA_Reports.csv.zip
- Outcomes: https://api.gdc.cancer.gov/data/1b5f413e-a8d1-4d10-92eb-7c4ae739ed81

Then execute the following code

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sksurv.nonparametric import kaplan_meier_estimator

In [ ]:
assert os.path.isfile('data/TCGA_Reports.csv'), 'Reports not extractected or downloaded'
assert os.path.isfile('data/TCGA-CDR-SupplementalTableS1.xlsx'), 'Outcomes not downloaded'

### Explore outcomes

In [ ]:
# Open outcomes
data = pd.read_excel('data/TCGA-CDR-SupplementalTableS1.xlsx', sheet_name = 'TCGA-CDR', index_col = 'bcr_patient_barcode')[['type', 'age_at_initial_pathologic_diagnosis', 'gender', 'race', 'ajcc_pathologic_tumor_stage', 'OS', 'OS.time']]

In [ ]:
# Open hospitals encoding - Used for the one hospital out (multiple analysis may happen in the same hospitals)
hospitals_encoding = pd.read_excel('data/TCGA-CDR-SupplementalTableS1.xlsx', sheet_name = 'TSS_Info', skiprows = [0, 1], index_col = 0)
hospitals_encoding.index = hospitals_encoding.index.astype(str)

In [ ]:
# Format data
data = data.rename(columns = {'OS': 'e', 'OS.time': 't'})
data.t = data.t / 365 # Convert days to years

data['Hospital'] = data.index.to_series().apply(lambda x: x[5:7]).replace({'NA': np.nan})
data = data.dropna()

data = data.replace({'[Not Available]': np.nan, '[Not Evaluated]': np.nan, '[Unknown]': np.nan, '[Discrepancy]': np.nan, '[Not Applicable]': np.nan})

data.gender = data.gender == 'MALE'
data.race = data.race == 'WHITE'
data.ajcc_pathologic_tumor_stage.loc[data.ajcc_pathologic_tumor_stage.str.contains('III', na = False)] = 3
data.ajcc_pathologic_tumor_stage.loc[data.ajcc_pathologic_tumor_stage.str.contains('II', na = False)] = 2
data.ajcc_pathologic_tumor_stage.loc[data.ajcc_pathologic_tumor_stage.str.contains('I', na = False)] = 1
data.ajcc_pathologic_tumor_stage.loc[data.ajcc_pathologic_tumor_stage.str.contains('X', na = False)] = np.nan

In [ ]:
# Hopsitals cleaning (lot of naming is repeated)
hospitals_encoding['Source Site'] = hospitals_encoding['Source Site'].str.lower()
short_list = pd.Series(hospitals_encoding['Source Site'].unique())
for hospital in short_list:
    hospitals_encoding['Source Site'].replace({to_replace: hospital for to_replace in short_list[short_list.str.contains(hospital)]}, inplace = True)

In [ ]:
# Change hospital encoding and remove small hospitals
data.Hospital = hospitals_encoding.loc[data.Hospital]['Source Site'].values

In [ ]:
# Look at the outcome distributions (survival outcome non-parametric display)
for type, count in data.type.value_counts().items():
    data_type = data[data.type == type]
    time, survival_prob = kaplan_meier_estimator(data_type.e.astype(bool), data_type.t / 365.)
    plt.step(time, survival_prob, where="post", label = type + ' (n = {})'.format(count), ls = ':', alpha = 0.75)
time, survival_prob = kaplan_meier_estimator(data.e.astype(bool), data.t / 365.)
plt.step(time, survival_prob, where="post", label = "Average", alpha = 0.75)
plt.ylim(0, 1)
plt.legend()
plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$ in years")

In [ ]:
# Group cancer types
grouping = {
    "Gastrointestinal": ['COAD', 'READ', 'ESCA', 'STAD', 'PAAD', 'CHOL', 'LIHC'],
    "Gynecological": ['BRCA', 'CESC', 'OV', 'UCEC', 'UCS'],
    "Genitourinary": ['KICH', 'KIRC', 'KIRP', 'PRAD', 'TGCT'],
    "Respiratory": ['LUAD', 'LUSC'],
    "Skin": ['ACC', 'HNSC', 'SKCM', 'UVM'],
    "Brain": ['GBM', 'LGG'],
    "Other": ['DLBC', 'MESO', 'SARC', 'THCA', 'THYM', 'PCPG', 'LAML', 'BLCA']
}

In [ ]:
# Display the new mean survival
data['Grouping'] = data.type.replace({vi: k for k, v in grouping.items() for vi in v})
for type in sorted(data.Grouping.unique()):
    data_type = data[data.Grouping == type]
    time, survival_prob = kaplan_meier_estimator(data_type.e.astype(bool), data_type.t / 365.)
    plt.step(time, survival_prob, where="post", label = type + ' (n = {})'.format(len(data_type)), ls = ':', alpha = 0.75)
time, survival_prob = kaplan_meier_estimator(data.e.astype(bool), data.t / 365.)
plt.step(time, survival_prob, where="post", label = "Average", alpha = 0.75)
plt.ylim(0, 1)
plt.legend()
plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$ in years")

### Explore reports

In [ ]:
reports = pd.read_csv('data/TCGA_Reports.csv')
reports.index = reports.patient_filename.str.split('.').apply(lambda x: x[0])
reports = reports.drop(columns = 'patient_filename')

In [ ]:
# Remove reports with no data and join with data
reports = reports.dropna().join(data, how = 'inner')
data = data.loc[reports.index.values]

In [ ]:
reports.to_csv('data/TGCA_Merged.csv')

In [ ]:
reports